# Machine learning for computational materials science and chemistry with MALA

## What is MALA?

A short summary about MALA and how it works is given in `mala_background.pdf`.

## Setting up MALA

For this tutorial, a Google Collab enviroment will be provided that includes all necessary packages. Generally, MALA is an open source framework that can be obtained [here](https://github.com/mala-project/mala). Detailled (installation) instructions can be found [here](https://mala-project.github.io/mala/).

A few examples at the end of the notebook tackle advanced applications. The necessary backends are, for the ease of installation, not bundled with the Google Collab environment. Interested readers may install them themselves on their machines, for in presence workshops, they will be demonstrated by the host, as are sosme aspects of data generation.

## Loading the modules

These modules will be necessary for the tutorials discussed here.

In [ ]:
# MALA itself.

import mala

# We would like to visualize simple plots.
# The font size can sometimes be a bit small for Jupyter Notebooks.

import matplotlib
import matplotlib.pyplot as plt
font = {'size'   : 22}
matplotlib.rc('font', **font)

# For the data paths.
from os.path import join as pj

# Only for the prediction down below.
from ase.io import read

# To do some timings down below.
from time import time


# Data I: Performing simulations (presentation only)

(See presenter screen)

# The MALA interface (hands-on)

Within extended ML frameworks, a big problem is reproducibility. Models often depend on a lot of so called hyperparameters, that characterize model behavior. This may include, but in no way be limited to, neural network layer sizes and number, training procedures, description of data specifics (e.g.: how is the local density of states sampled?), etc.
There is a number of ways to efficiently handle this problem. A lot of frameworks rely on command line arguments to deal with this, i.e., the user provides a, potentially extensive, list of command line arguments upon runtime. Another good way to handle all this is the usage of input files. This is consistent with the way standard computational science simulation codes work.
An obvious downside to this is that one has to have a framework at hand to prepare these input files.

MALA follows a route sort of in between the two approaches. The central quantity is the `Parameters()` object. It holds ALL (hyper)parameters one could use in the course of a MALA run. It is structured by the subtasks of MALA.

In [ ]:
parameters = mala.Parameters()

# All parameters related to how data is handled in general.
parameters.data

# "Targets" always refers to the quantity being learned.
parameters.targets

# "Descriptors" always refers to the quantity from which we learn.
parameters.descriptors

# "Data generation" refers to useful routines for creating training data.
# These routines are mostly experimental at the moment and will not be discussed here in detail
parameters.datageneration

# "Network" refers to everything related to neural network creation and training.
# In the future, support for more models is planned, and this collection of parameters
# will be updated to reflect this.
parameters.network

# "Hyperparameters" means hyperparameter optimization. This is the process of finding the optimal
# hyperparameters for MALA model training, and we will come back to this process later.
parameters.hyperparameters

Individual parameter objects can be printed to see what's hidden inside. This also works on the main object as well.

Try printing a different parameter subset.

In [ ]:
parameters.data.show()

Finally, there are some high-level parameters one needs when performing ML at scale.

In [ ]:
# Whether or not to use a GPU for model training and inference.
parameters.use_gpu

# Whether or not to use MPI parallel CPU inference (no training supported).
# This option is either for pre-processing or production runs of trained models.
# More on that later.
parameters.use_mpi

# Manual seeds can be used to fix the Pseudo RNG to re-create models with the exact
# same model weights.
parameters.manual_seed

# A comment may be useful to distinguish between sets of parameters.
parameters.comment

# This is useful for adjusting the output level of MALA.
parameters.verbosity

All of this does not explain how MALA handles reproducibility. Write hundreds of lines of parameter statement in each python script is not exactly maintainable.
Therefore MALA provides a .json interface.

In [ ]:
parameters.save(...)
# Loading can be done via Parameters.load_from_file(...)

Have a look at the .json file that was just created. You will see that it is structured in the same ways as the python object, allowing fast access. You will see some parameters that are not dicussed here since they exceed the scope of this tutorial. For a first excercise, try to modify parameters both in python and in json and see whether loading will recover those changes. The comment and manual seed are good first examples for this.

In [ ]:
# Set a comment
...

# Save.
...

In [ ]:
# Edit something in the file (e.g. the manual_seed), reload and print.
...

For extended experiments, it is very useful to operate with such input files and only use the in-python parameter editing when absolutely necessary. Further, concluded experiments can be saved in this way for future reference.
In the following, we will use a combination of both approaches for the sake of transparency.

# Data II: Data preprocessing (presentation-only)

(See presenter screen)

In [ ]:
data_path = ??? # TODO: Check how this works with Google Collab.

# Visualizing and reproducing output data (hands-on)

Before we train a model, it is a good idea to think about which metric is important, i.e., how do we test if a model is good?

In essence, the advantage of MALA is the access to multiple observables. Two easily accesible metrics are the density of states (DOS) and the band energy. We will now learn how to calculate them from the LDOS (the actual DFT LDOS in this case) so we can do the same after model training to test our models.

For this, we first have to make sure the correct LDOS parameters are used.

In [ ]:
parameters.targets.target_type = "LDOS"
parameters.targets.ldos_gridsize = 11
parameters.targets.ldos_gridspacing_ev = 2.5
parameters.targets.ldos_gridoffset_ev = -5

Now we can create an LDOS calculator and directly populate it with the LDOS data from the data set.

In [ ]:
ldos_calculator = mala.LDOS.from_numpy_file(parameters, pj(data_path, "Be_snapshot0.out.npy"))

Afterwards, we have to read in some additional information from the simulation data (size of the real space grid, temperature, etc.).

In [ ]:
ldos_calculator.read_additional_calculation_data(pj(data_path, "Be_snapshot0.out"))

Now we can access the DOS and the band energy as properties of the calculator object.

In [ ]:
# Electronic structure data is provided as properties of calculator object,
# i.e. .energy_grid (x-axis) and .density_of_states (y-axis).
# Band energy can be accessed the same way.
# Plot with e.g. matplotlib.

# Training a model (hands-on)

Finally, we can train a neural network based model for the electronic structure. First, let us review which parameters we need for this. In the following we will slowly adapt the parameters until we get a good model out of it.

Since we want to learn about the inner workings of MALA, we want full output. We will also fix the manual seed, so that all the models are comparable.

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2

# Choose whatever you like.
parameters.manual_seed = ...

Since MALA provides quite a few reasonable default values, in the simplest case the only thing we have to decide upon is the data we want to learn on and the architecture of the neural network.

For each training we have to specify training (`"tr"`) and validation (`"va"`) snapshots. The former are used to actually tune the network weights, the latter monitor model performance during training. They will become very relevant as we optimize the training process.

Deciding on the layer sizes is usually done AFTER the data is loaded, since the first and last layer need to match up with the data provided.

In [ ]:
data_handler = mala.DataHandler(parameters)
data_handler.add_snapshot(input_file_name, path_to_input_file,
                          output_file_name, path_to_output_file, "tr")
# What about validation data?

# This already loads data into RAM!
data_handler.prepare_data()

# This statement is best kept AFTER the data handling
parameters.network.layer_sizes = [data_handler.input_dimension,
                                  100,
                                  data_handler.output_dimension]

Now we can actually train a network.

In [ ]:
network = mala.Network(parameters)
trainer = mala.Trainer(parameters, network, data_handler)
trainer.train_network()

Well, how was that? Do we have a good model now, can we predict the LDOS with this? That is not an easy question to answer from this output alone. First of all, we see loss values being printed, and those look all nice, but they are not trivially related to physical/chemical accuracy, which we are actually looking for.

We can test this by using the `Tester` class. The class works similar to the Trainer class. We add data, push them through the model, and then use the results to perform calculations.

We just have to make sure that the LDOS is correctly integrated by setting the appropriate parameters. Then we can add data to test. We should always test on data different from the one we trained on. Also, we now have to specify the corresponding calculation output, since we may need this for integration.

In [ ]:
parameters.targets.ldos_gridsize = ...
parameters.targets.ldos_gridspacing_ev = ...
parameters.targets.ldos_gridoffset_ev = ...

# The data handler still holds the data from training.
# What happens when we instead create a new one?
# Try adding more then one snapshot.
# We have four in total, trained on one, validated on one,
# that leaves one for testing.
data_handler.clear_data()
data_handler.add_snapshot(input_file_name, path_to_input_file,
                          output_file_name, path_to_output_file, what_goes_here?,
                          calculation_output_file=what_is_this?)

# What happens when you select "True" here?
data_handler.prepare_data(reparametrize_scaler=False)


Now comes the actual object with which to test. We simply tell it which observables to test for and off we go. The results are given as a dictionary and in the units of meV/atom.

In [ ]:
# Other observables could be "number_of_electrons" or "total_energy". Feel free to try them out!
tester = mala.Tester(parameters, network, data_handler, observables_to_test=["band_energy"])
results = tester.test_all_snapshots()
print(results)

That already looks pretty solid. On the machine this notebook was tested on, errors below 0.1 meV/atom were reported, i.e., the network reproduces the band energy excellently. We may also visualize the DOS, and will do so later, but for now let's focus on the band energy. Can we get even better results? Of course, for simple data like this, we already seem to be doing a good job. But larger and more diverse data sets can be tricky.

So let us see if we can improve the training routine even further. To do so, first let's rewrite the code above into functions so that we can toy around with them.

In [ ]:
def training(parameters):
    ...

    # Beware that the network layers still have to be treated after data loading!

    return parameters, data_handler, network

def testing(parameters, data_handler, network):
    ...

    return results


Now let's see what happens if we start changing up the parameters. Let's start with those that influence the network architecture first. If we define the functions in an appropriate way, we can easily manipulate layer sizes of the neural networks for instance. What happens if we choose a ridiculously small number of neurons for the hidden layer? What happens if we choose a large number? Try e.g. 2 or 1000 as number for the hidden layer (the one in between).

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = ...
parameters.network.layer_sizes = [...]

parameters, data_handler, network = training(parameters)
print(testing(parameters, data_handler, network))


Ok, so this clearly has an effect. We can build better and worse models by simply adjusting this number. Models that are too small don't have enough information capacity - they simply cannot capture the information present in the data. This is called **underfitting**.

But also big models (if you haven't tried - try e.g. 1000 neurons!) do not yield immediate improvement. In my case, 1000 neurons in the middle gave noticeably worse performance.

Why is that? Large models possess a large capacity to store information. If they are not carefully fitted, they may very soon begin to **overfit**, i.e. learn to predict the training data - and only the training data. This is obviously not what is supposed to happen.

This is what the aforementioned validation data is for. Validation data is data unseen by the model during the weight optimization, but checked after each epoch. The idea is simple: Since the model is not directly fitted on the validation data, we can use it to monitor fitting progress. If validation accuracy is drastically different from training accuracy, something is off.

If you check your outputs, you will realize that both losses behave differently for different models. In the 1000 neuron case at the end you will encounter a factor of 10 between both of them! For a smaller model, it was only 2.

There is one way to alleviate this: Early stopping. The idea is to end training prematurely if we see validation accuracy stagnating for a certain amount of time, i.e., the model starting to overfit. MALA supports this with a simple command. Let's try it out with 1000 neurons.

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = ...
parameters.network.layer_sizes = [...]
parameters.running.early_stopping_epochs = ...

parameters, data_handler, network = training(parameters)
print(testing(parameters, data_handler, network))


In my case, that helped a tiny bit with the accuracy of one snapshot. So we're getting there. Before we overthinking this specific aspect, let's introduce some more hyperparameters and test them out. We can toy with the length of the training, the learning rate (i.e. the aggressivness of the gradient updates of the neural networks, large learning rates mean larger updates per epoch, which can lead to oscillations, small learning rates can lead to stagnation) and the optimizer being used.

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = ...
parameters.network.layer_sizes = [...]
parameters.running.early_stopping_epochs = ...
# Default value is 100 - try out longer trainings as well. Or shorter!
parameters.running.max_number_epochs = 100
# Default value is 0.5 - try out a few!
parameters.running.learning_rate = 0.5
# Default value is SGD - try out Adam instead!
parameters.running.trainingtype = "SGD"

parameters, data_handler, network = training(parameters)
print(testing(parameters, data_handler, network))


You may beginning to feel a bit overwhelmed by all the choices one can make here. We'll address this very real struggle in a second, but we'll just add a bit more gasoline to the fire while were at it.

There are two aspects of ML that can be very important (and that MALA supports) that we have not yet talked about.

The first is data scaling. The idea behind data scaling is to make data more uniform - vectorial fields as in our case may contain larger and smaller components per data point and it can be hard for a network to correctly learn this. By normalizing or standardizing data, performance is often improved. MALA supports normalization feature-wise or per entire dataset. Empirically, we have made good experiences with the following. (Be aware: if you change the scaling, the absolute loss values will inevitably change - this is why we monitor the band energy.)

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = 2023
parameters.network.layer_sizes = [100]

# Options are "None", "normal" and "standard". The two latter ones can alsoe
# be used in combination with the phrase "feature-wise-".
parameters.data.input_rescaling_type = "feature-wise-standard"
parameters.data.output_rescaling_type = "normal"


parameters, data_handler, network = training(parameters)
print(testing(parameters, data_handler, network))


In my case, adding the data scaling improved performance drastically. Feel free to try out other forms of scaling (for no scaling, just specify `"None"`).

Secondly, we can adapt the activation functions of the neural net, i.e., the linear transformation at the end of each layer. In MALA, this is realized by a list as well. We can specify individual layers, or simply give only one type to be used at each layer.

In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = 2023
parameters.network.layer_sizes = [100]
parameters.data.input_rescaling_type = "feature-wise-standard"
parameters.data.output_rescaling_type = "normal"

# Sigmoid is default, try out ReLU or LeakyReLU!
parameters.network.layer_activations = ["Sigmoid"]


parameters, data_handler, network = training(parameters)
print(testing(parameters, data_handler, network))


As you will have noticed, the result seem to vary quite a bit.

So what do we make of all of this?
We can build different models, and they give different accuracy. But that's also dependent on the data scaling to be used. And that is also dependent on the way we train them. What now?

Well, welcome to hell. This complexity is what makes (deep) ML hard, just as the obvious speed-up makes it worthwhile.

There are two important concepts at play here:

1. Intuition and experience - understanding your data. Your data has inherent structure that can be exploited in several ways. As a rule of thumb activation functions may be chosen in a way that reflects the data itself, i.e. if we have a lot of smooth curves, Sigmoid functions may work well (they do here). LeakyReLU on the other hand is a standard choice for more complicated data. Model size should reflect the complexity in your data set, and a simple data set does not warrant a huge model. We have seen this just now. Training routines should be optimized for the model.
2. Good, old fashioned non-linear optimization. Essentially we have a N-dimensional hypersphere with some distinct or float valued choices and can simply test out all the combinations.

The best possible route is a combination of the two. By adding as much prior experience from 1. into 2. we can perform a limited optimization and still end up with the best possible model.

# Tuning the Hyperparameters (hands-on)

MALA provides a convenient way to tune the hyperparameters of models. There are multiple algorithms implemented, but we will focus on the `optuna` library, to which MALA provides an interface. The idea is easy: we give MALA a set of hyperparameters to optimize, fix the others, give it some data and let optuna do its work. Optuna internally uses elaborate algorithms to determine optimal hyperparameters from observed data.

First, let us think about which hyperparameters to fix. We want to fix as many as possible and reasonable to get the best trade-off between optimal results and minimal computational effort.


In [ ]:
parameters = mala.Parameters()
parameters.verbosity = 2
parameters.manual_seed = 2023
parameters.data.input_rescaling_type = "feature-wise-standard"
parameters.data.output_rescaling_type = "normal"
parameters.running.max_number_epochs = 100
parameters.running.mini_batch_size = 40
parameters.running.trainingtype = "SGD"

Now we can specify data to be used.


In [ ]:
data_handler = mala.DataHandler(parameters)
...

Before we instantiate the hyperparameter optimizer, two important parameters to set are

1. The number of trials (test networks) to train
2. How often to train a test network - if we train each proposed network a number of times and evaluate the average performant, we can discard unrobust outliers. In the interest of time, let's keep it at 1 here.

In [ ]:
parameters.hyperparameters.n_trials = 20
parameters.hyperparameters.number_training_per_trial = 1

hyperoptimizer = mala.HyperOptOptuna(parameters, data_handler)

hyperoptimizer.add_hyperparameter("categorical", "learning_rate",
                                     choices=[0.1, 0.2, 0.5])
# Which other hyperparameters should we maybe add?


The syntax is a bit clunky at times, since we are trying to cramp a complicated system into a few function calls. Now let's run. This may take a while. Grab a coffee and sit back :)


In [ ]:
hyperoptimizer.perform_study()

# Will save the results directly to the parameters.
hyperoptimizer.set_optimal_parameters()


Let's train the best network one final time and see where we stand. In my case, the network that Optuna suggest is bigger and deeper then the ones we have used before, but that may differ. Keep in mind we are performing a very
 limited search here in the interest of time.

Also, please keep in mind that the identified network already includes a first and last layer.

In [ ]:
...
trainer.train_network()
print(testing(parameters, data_handler, network))

The results may not necessarily be better with this limited search then what we had above. In my case they are very good, but again, this is a limited search compared to reasonable defaults. For actual production runs, one would set up a thorough, longer search before training production level models.

# Excurse: How to tune bispectrum hyperparameters (presentation only)


(See presenter screen)


In [ ]:
acsd_parameters = mala.Parameters()
acsd_parameters.descriptors.descriptor_type = "Bispectrum"
acsd_parameters.descriptors.bispectrum_twojmax = 10
acsd_parameters.descriptors.bispectrum_cutoff = 4.67637
acsd_parameters.descriptors.descriptors_contain_xyz = True
acsd_parameters.targets.target_type = "LDOS"
acsd_parameters.targets.ldos_gridsize = 11
acsd_parameters.targets.ldos_gridspacing_ev = 2.5
acsd_parameters.targets.ldos_gridoffset_ev = -5
acsd_parameters.descriptors.descriptors_contain_xyz = True
acsd_parameters.descriptors.acsd_points = 100

hyperoptimizer = mala.ACSDAnalyzer(acsd_parameters)
hyperoptimizer.add_hyperparameter("bispectrum_twojmax", [2, 4, 10])
hyperoptimizer.add_hyperparameter("bispectrum_cutoff", [0.5, 2.0, 4.67637])

# Add raw snapshots to the hyperoptimizer. For the targets, numpy files are
# okay as well.
hyperoptimizer.add_snapshot("espresso-out", pj(data_path, "Be_snapshot1.out"),
                            "numpy", pj(data_path, "Be_snapshot1.in.npy"),
                            target_units="1/(Ry*Bohr^3)")
hyperoptimizer.add_snapshot("espresso-out", pj(data_path, "Be_snapshot2.out"),
                            "numpy", pj(data_path, "Be_snapshot2.in.npy"),
                            target_units="1/(Ry*Bohr^3)")

# If you plan to plot the results (recommended for exploratory searches),
# the optimizer can return the necessary quantities to plot.
plotting = hyperoptimizer.perform_study()
hyperoptimizer.set_optimal_parameters()


# Inference and prediction (hands-on / presentation)

We now understand how we can build optimal models with MALA. Once this is done, we would like to use these models to predict properties of interest. For this, let us first see how MALA can be used to save and load models.

We train a final model - based on the parameters idenitified above.


In [ ]:
...
trainer.train_network()


Saving such a model now becomes a one-liner. Models will be saved as `.zip` archives containing the network weights, parameter json file and coefficients for the data scaling.

In [ ]:
trainer.save_run("Be_model")

Loading is equally simple, but dependent on the task. Two possible task can be done: Testing (as we have done above, to verify whether a model is performing well) or predicting (using the model to predict the electronic structure of arbitrary atomic configurations).

We will have a quick look at the testing interface again to plot the density of states, and see how MALA can directly give us access to the electronic structure of a material.


In [ ]:
parameters, network, data_handler, tester = mala.Tester.load_run("Be_model")
...
# Can you predict for more then one snapshot at a time?
actual_ldos, predicted_ldos = tester.predict_targets(0)


Now we can visualize the DOS via the internal LDOS calculator of the DataHandler object.

In [ ]:
ldos_calculator = data_handler.target_calculator
ldos_calculator.read_additional_calculation_data(data_handler.get_snapshot_calculation_output(0))
# In the same way we had used "from_numpy_file" above, we can read the LDOS from memory with
# "read_from_array"
...


This finally leads us to the prediction part - the part, to which such efforts eventually should culminate too. Since this part again requires LAMMPS, it will not be hands-on.

Please see the presenter screen for the prediction part.

# Making MALA performant (presentation only)

(See presenter screen)
